In [27]:
# coding:utf8
import requests, json

In [13]:
# 获取输入的股票信息
def get_stock_with_input():

    code_name = input("输入查询的股票名称及代码，如'招商银行-22600036.SH,'")
    stock_code = ''
    stock_name = ''
    if code_name == '':
        stock_code = '22600036.SH'
        stock_name = '招商银行'
    else:
        stock_name = code_name.split('-')[0]
        stock_code = code_name.split('-')[1]

    print('股票名称:',stock_name)
    print('股票代码:',stock_code)

    return (stock_name, stock_code)

In [15]:
stock_name, stock_code = get_stock_with_input()
stock_name

股票名称: 招商银行
股票代码: 22600036.SH


'招商银行'

In [42]:
def get_north_with_stock_code(stock_code):
    url = 'https://datacenter.eastmoney.com/securities/api/data/get?type=RPT_MUTUAL_STOCK_HOLDRANKN&sty=SECURITY_CODE,SECUCODE,SECURITY_NAME,TRADE_DATE,IS_ADJ_DATE,HOLD_MARKET_CAP,HOLD_SHARES,FREE_SHARES_RATIO,TOTAL_SHARES_RATIO,PARTICIPANT_NUM,ADD_MARKET_CAP,ADD_SHARES_REPAIR,ADD_SHARES_AMP,CLOSE_PRICE,CHANGE_RATE,MUTUAL_TYPE&callback=&extraCols=&filter=(INTERVAL_TYPE="1")(MUTUAL_TYPE="001")(SECUCODE="{}")&p=1&ps=100&sr=-1&st=TRADE_DATE&token=&var=&source=DataCenter&client=APP'.format(stock_code)
    response = requests.get(url)
    print(json.dumps(response.json(), indent=4))
    if response.status_code == 200:
        response_json = response.json()
        if response_json['success'] is True:
            return response_json['result']['data']
    else:
        print('数据请求失败，请重试.')



In [43]:
north_arr = get_north_with_stock_code('600309.SH')
print(north_arr)

ES_RATIO": 4.0051,
                "PARTICIPANT_NUM": 58,
                "ADD_MARKET_CAP": 49668915.7723,
                "ADD_SHARES_REPAIR": 546197,
                "ADD_SHARES_AMP": 0.43624402,
                "CLOSE_PRICE": 90.39,
                "CHANGE_RATE": 0,
                "MUTUAL_TYPE": "001"
            },
            {
                "SECURITY_CODE": "600309",
                "SECUCODE": "600309.SH",
                "SECURITY_NAME": "\u4e07\u534e\u5316\u5b66",
                "TRADE_DATE": "2020-11-17 00:00:00",
                "IS_ADJ_DATE": "\u5426",
                "HOLD_MARKET_CAP": 11317231772.13,
                "HOLD_SHARES": 125204467,
                "FREE_SHARES_RATIO": 8.794,
                "TOTAL_SHARES_RATIO": 3.9877,
                "PARTICIPANT_NUM": 59,
                "ADD_MARKET_CAP": -6816303.2952,
                "ADD_SHARES_REPAIR": -75282,
                "ADD_SHARES_AMP": -0.06009112,
                "CLOSE_PRICE": 90.39,
                "CHANGE_

AttributeError: 'dict' object has no attribute 'success'